In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

import json
import pickle
import logging

import pandas as pd
import numpy as np

from itext2kg import iText2KG
from itext2kg.utils import PubtatorProcessor
from langchain_ollama import ChatOllama, OllamaEmbeddings
from itext2kg.graph_integration import GraphIntegrator

/home/mindrank/miniconda3/envs/kg/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function array> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [4]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [5]:
NEO4J_URI = "neo4j+s://435eb4f0.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "nqL3At3rptFpfRrBhg5Y1-7k78oHqVaW42WKtqVRnJo"

llm = ChatOllama(
    model="deepseek-r1:32b",
    temperature=0,
)

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

In [6]:
PUTATOR_PATH = "/home/mindrank/fuli/itext2kg/Data/delirium"
JSON_PATH = "/home/mindrank/fuli/itext2kg/output_kg/"
PMID = 19361117

In [7]:
with open(f"{PUTATOR_PATH}/{PMID}.txt", "r") as f:
    text = f.readlines()

In [8]:
text

['19361117|t|Alternative comorbidity adjustors for the Medicare inpatient psychiatric facility PPS.\n',
 '19361117|a|The inpatient psychiatric facility prospective payment system (IPF-PPS), provides per diem payments for psychiatric hospitals and units, including 17 comorbid condition payment adjustors that cover 11 percent of patients. This study identifies an alternative set of 16 adjustors identifying three times as many high-cost patients and evaluates the improved predictive power in log per diem cost regression models. A model using the IPF-PPS adjustors achieved 8.8 percent of the feasible improvement from a no-adjustor baseline, while the alternative adjustors achieved 22.1 percent of the feasible improvement. The current adjustors may therefore be too restrictive, resulting in systematic over- or underpayment for many patients.\n',
 '19361117\t51\t60\tinpatient\tSpecies\t\n',
 '19361117\t61\t72\tpsychiatric\tDisease\tMESH:D001523\n',
 '19361117\t91\t100\tinpatient\tDisease\t\n

In [9]:
pubtator_file = f"{PUTATOR_PATH}/{PMID}.txt"
pubtator_process = PubtatorProcessor(pubtator_file, llm)
semantic_blocks = pubtator_process.block
properties_info = pubtator_process.properties_info
pubtator_info = pubtator_process.pubtator_info

2025-03-06 22:20:06,694 - INFO - Processing PubTator file: /home/mindrank/fuli/itext2kg/Data/delirium/19361117.txt
2025-03-06 22:20:06,695 - WARNING - Skipping malformed entity line: ['']


2025-03-06 22:21:54,111 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-06 22:22:00,239 - ERROR - Attempt 1 failed: Error distilling abstract: peer closed connection without sending complete message body (incomplete chunked read)
Traceback (most recent call last):
  File "/home/mindrank/miniconda3/envs/kg/lib/python3.11/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/home/mindrank/miniconda3/envs/kg/lib/python3.11/site-packages/httpx/_transports/default.py", line 127, in __iter__
    for part in self._httpcore_stream:
  File "/home/mindrank/miniconda3/envs/kg/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 407, in __iter__
    raise exc from None
  File "/home/mindrank/miniconda3/envs/kg/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 403, in __iter__
    for part in self._stream:
  File "/home/mindrank/miniconda3/envs/kg/lib/python3.11/site-packages/ht

TypeError: sequence item 1: expected str instance, NoneType found

In [ ]:
properties_info

{'source': 'PMID31974315', 'species': '1578,10090'}

In [ ]:
semantic_blocks

["disease - [['disease': 'postoperative delirium'], ['disease': 'postoperative delirium'], ['disease': 'neurological disorders'], ['disease': 'mitochondrial dysfunction']]",
 "protein - [['protein': 'IL-6'], ['protein': 'PSD-95'], ['protein': 'synaptophysin']]",
 "region - [['region': 'brain']]",
 "gene - [['gene': 'IL-6'], ['gene': 'PSD-95'], ['gene': 'synaptophysin']]",
 "chemical - [['chemical': 'isoflurane']]",
 'Title: Anesthesia and surgery induce age-dependent changes in behaviors and microbiota.\n Abstract: The neuropathogenesis of postoperative delirium remains mostly unknown The gut microbiota is implicated in the pathogenesis of neurological disorders We therefore set out to determine whether anesthesiasurgery causes age-dependent gut microbiota dysbiosis changes in brain IL-6 level and mitochondrial function leading to postoperative delirium-like behavior in mice Female 9 or 18 months old mice received abdominal surgery under 14% isoflurane for two hours The postoperative d

In [ ]:
pubtator_info['abstract'] = {'context':semantic_blocks[-1], 'source':properties_info['source']}

In [ ]:
pubtator_info

{'postoperative delirium': {'label': 'disease',
  'unique_id': 'MESH:D000071257'},
 'neurological disorders': {'label': 'disease', 'unique_id': 'MESH:D009461'},
 'IL-6': {'label': 'gene', 'unique_id': 'Gene ID:16193'},
 'isoflurane': {'label': 'chemical', 'unique_id': 'MESH:D007530'},
 'PSD-95': {'label': 'gene', 'unique_id': 'Gene ID:13385'},
 'synaptophysin': {'label': 'gene', 'unique_id': 'Gene ID:20977'},
 'mitochondrial dysfunction': {'label': 'disease',
  'unique_id': 'MESH:D028361'},
 'abstract': {'context': 'Title: Anesthesia and surgery induce age-dependent changes in behaviors and microbiota.\n Abstract: The neuropathogenesis of postoperative delirium remains mostly unknown The gut microbiota is implicated in the pathogenesis of neurological disorders We therefore set out to determine whether anesthesiasurgery causes age-dependent gut microbiota dysbiosis changes in brain IL-6 level and mitochondrial function leading to postoperative delirium-like behavior in mice Female 9 or

In [ ]:
# -----------------Build the graph-----------------
itext2kg = iText2KG(llm_model = llm, embeddings_model = embeddings)
kg = itext2kg.build_graph(
    sections=[semantic_blocks], 
    source=properties_info,  
    entities_info=pubtator_info,
    ent_threshold=0.9, 
    rel_threshold=0.4,
    )

2025-03-06 19:12:09,504 - INFO - Extracting Entities from the Document


2025-03-06 19:12:09,892 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-06 19:12:27,139 - INFO - INFO --- postoperative delirium add unique_id and label
2025-03-06 19:12:27,141 - INFO - INFO --- neurological disorders add unique_id and label
2025-03-06 19:12:27,142 - INFO - INFO --- mitochondrial dysfunction add unique_id and label
2025-03-06 19:12:27,143 - INFO - INFO --- il-6 add unique_id and label
2025-03-06 19:12:27,143 - INFO - INFO --- psd-95 add unique_id and label
2025-03-06 19:12:27,144 - INFO - INFO --- synaptophysin add unique_id and label
2025-03-06 19:12:27,145 - INFO - INFO --- il-6 add unique_id and label
2025-03-06 19:12:27,145 - INFO - INFO --- psd-95 add unique_id and label
2025-03-06 19:12:27,146 - INFO - INFO --- synaptophysin add unique_id and label
2025-03-06 19:12:27,147 - INFO - INFO --- isoflurane add unique_id and label
2025-03-06 19:12:27,147 - INFO - {'entities': [{'label': 'disease', 'name': 'postoperative delirium', '

In [ ]:
GraphIntegrator(uri=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD).visualize_graph(knowledge_graph=kg)

2025-03-06 19:12:56,208 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (m))} {position: line: 1, column: 1, offset: 0} for query: 'MATCH (n:chemical {name: "isoflurane"}), (m:disease {name: "postoperative delirium"}) MERGE (n)-[r:used_in_study_of]->(m) SET r.embeddings = [-0.0032269114, 0.012367626, -0.08854565, -0.05865524, 0.019571966, -0.028894896, -0.010526052, 0.007606421